In [1]:
import tensorflow as tf

import pandas

data = pandas.read_table('data/adult.data', skipinitialspace=True, sep=',')

data.shape

(32561, 15)

In [2]:
data.iloc[:5, :7]

,age,workclass,fnlwgt,education,education-num,marital-status,occupation
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty


In [3]:
data.iloc[:5, 7:]

,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,Husband,White,Male,0,0,13,United-States,<=50K
2,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,Husband,Black,Male,0,0,40,United-States,<=50K
4,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
null = data.isnull().sum()

null.sum()

0

In [5]:
from sklearn.model_selection import train_test_split

from util import add_extra_rows

train, test = train_test_split(data, test_size=0.2, shuffle=False)

test_sample = add_extra_rows(test)

test.shape[0], test_sample.shape[0]

(6513, 6517)

`Table Statistics`

In [6]:
from tensorflow_data_validation import generate_statistics_from_dataframe

train_stats = generate_statistics_from_dataframe(train)

statlist = {}

statlist['rhs_statistics'] = train_stats

statlist['rhs_name'] = 'Train Stats'

type(train_stats)

tensorflow_metadata.proto.v0.statistics_pb2.DatasetFeatureStatisticsList

In [7]:
from tensorflow_data_validation import visualize_statistics

visualize_statistics(train_stats)

In [8]:
test_stats = generate_statistics_from_dataframe(test_sample)

statlist['lhs_statistics'] = test_stats

statlist['lhs_name'] = "Test Stats"

type(test_stats)

tensorflow_metadata.proto.v0.statistics_pb2.DatasetFeatureStatisticsList

In [9]:
visualize_statistics(**statlist)

`Set Lower & Upper Limit Age Data on Test Sample Table`

`0 to 17`

`100 to 90`

In [10]:
lower_age = test_sample['age'] > 16

upper_age = test_sample['age'] < 91

test_sample = test_sample[lower_age & upper_age]

test_sample = test_sample.dropna()

In [11]:
test_stats = generate_statistics_from_dataframe(test_sample)

statlist['lhs_statistics'] = test_stats

statlist['lhs_name'] = "Test Stats"

visualize_statistics(**statlist)

`Table Schema`

In [12]:
from tensorflow_data_validation import infer_schema, display_schema

train_main_schema = infer_schema(train_stats)

display_schema(train_main_schema)

`Anomalies Checker`

`Validate Test Dataset on Train Schema (Main Schema)`

In [13]:
from tensorflow_data_validation import validate_statistics, display_anomalies

test_sample_anomalies = validate_statistics(test_stats, train_main_schema)

display_anomalies(test_sample_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'occupation',Unexpected string values,Examples contain values missing from the schema: gamer (<1%).
'race',Unexpected string values,Examples contain values missing from the schema: Asian (<1%).
'native-country',Unexpected string values,Examples contain values missing from the schema: Mongolia (<1%).


`ENUM_TYPE_UNEXPECTED_STRING_VALUES`